In [9]:
import pandas as pd
import numpy as np
import xlwings as xw
import glob

In [14]:
WEEK = "40th_2021"

PATH_TRANSFORMED_MVEXP = f"D:\\Run Analysis\\blob_storage\\truck_exp_other\\transformed_og_ds\\{WEEK}.csv"   

PATH_ROSTER = f"D:\\Run Analysis\\blob_storage\\roster\\{WEEK}.xlsx"

PATH_COMPLETE_ALLOC = f"D:\\Run Analysis\\blob_storage\\truck_exp_other\\processed\\{WEEK}_alloc.csv"

PATH_COMPLETE_DETAIL = f"D:\\Run Analysis\\blob_storage\\truck_exp_other\\processed\\{WEEK}_deatiled.csv"

mvexp_df = pd.read_csv(PATH_TRANSFORMED_MVEXP)

roster_df = pd.read_excel(PATH_ROSTER)

mvexp_df_trucks_list = mvexp_df['Job No.'].unique()

pri_trucks = roster_df.dropna(
subset=['Primary_truck']).Primary_truck.str.replace(' ', '').unique()

sec_trucks = roster_df.dropna(
subset=['Alternative_Truck']).Alternative_Truck.str.replace(' ', '').unique()

roster_df_trucks_list = np.append(pri_trucks, sec_trucks)

uni_ros_trucks = list(set(roster_df_trucks_list))

ros_trucks = np.array(uni_ros_trucks)

# Excel
trucks_not_in_yard = np.setdiff1d(
ros_trucks, mvexp_df_trucks_list, assume_unique=True)
# Excel
trucks_in_yard = np.setdiff1d(
mvexp_df_trucks_list, ros_trucks, assume_unique=True)

In [15]:
def clean_roster_df(df):
        df.Primary_truck = df.Primary_truck.str.replace(" ", "")
        df.Alternative_Truck = df.Alternative_Truck.str.replace(" ", "")
        df = df.fillna(0)
        return df

roster_df = clean_roster_df(roster_df)

pri_truck_runs_count = roster_df.Primary_truck.value_counts()

sec_truck_runs_count = roster_df.Alternative_Truck.value_counts()

In [18]:
total_truck_runs_count = (pd
                              .concat([pri_truck_runs_count, sec_truck_runs_count])
                              .reset_index()
                              .groupby('index')
                              .sum().iloc[1:]
                              .reset_index()
                              .rename(columns={"index": "number_plate", 0: "trucks"})

                              )

In [20]:
mvexp_truckcounts_df = pd.merge(
        mvexp_df, total_truck_runs_count, how="left", left_on='Job No.', right_on='number_plate')

In [26]:
mvexp_truckcounts_df['per_truck_cost'] = mvexp_truckcounts_df.Debit / \
        mvexp_truckcounts_df.trucks

mvexp_cost_summary_df = mvexp_truckcounts_df[[
        'Acc_name', 'Job No.', 'per_truck_cost']].dropna(subset=['per_truck_cost'])

In [56]:
mvexp_cost_summary_df

# truck_cost = pd.merge(roster_df, mvexp_cost_summary_df, how="left",left_on='Primary_truck', right_on='Job No.')

# total_truck_cost = pd.merge(truck_cost, mvexp_cost_summary_df, how="left", left_on='Alternative_Truck', right_on='Job No.')  

pri_truck_cost = pd.merge(roster_df, mvexp_cost_summary_df, how="left",left_on='Primary_truck', right_on='Job No.')

alt_truck_cost = pd.merge(roster_df, mvexp_cost_summary_df, how="left", left_on='Alternative_Truck', right_on='Job No.') 
PATH =  "D:\\Run Analysis\\blob_storage\\truck_exp_other\\truck_cost_3.csv"

truck_cost = pd.concat([pri_truck_cost, alt_truck_cost], ignore_index=True)

# .to_csv(PATH, index=False)  


In [57]:

def per_route_cost(df):
    count = 0 
    p_route = df.Primary_route
    route_1 = df.Satellite_Route_1
    route_2 = df.Satellite_Route_2
    
    if p_route != 0:
        count = count + 1
    if route_1 != 0:
        count = count + 1
    if route_2 != 0:
        routes = route_2.split("/")
        count = count + len(routes)

    cost = df.per_truck_cost / count
    
    return cost, count

In [59]:
truck_cost[['a','b']] = truck_cost.apply(per_route_cost, axis=1, result_type='expand')

In [61]:
PATH =  "D:\\Run Analysis\\blob_storage\\truck_exp_other\\truck_cost_2.csv"

truck_cost.to_csv(PATH, index=False)
